In [1]:
import ee 
import geemap
ee.Initialize()

## The map function

In [2]:
mylist= ee.List.sequence(1, 10)
mylist

In [6]:
def square(number):
    return ee.Number(number).pow(2)

squares= mylist.map(square)
squares

In [7]:
def square(number):
    return ee.Number(number).pow(2).subtract(1)

squares= mylist.map(square)
squares

In [8]:
## Lambda function function (only apply for one line function)

squares= mylist.map(lambda number: ee.Number(number).pow(2).subtract(1))
squares

### same in python without ee

In [3]:
for i in range(1,10):
    print(i)

1
2
3
4
5
6
7
8
9


In [4]:
squares= []
for i in range(1,11):
    squares.append(i**2)
squares

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [5]:
[i**2 for i in range(1,11)]

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [12]:
Map = geemap.Map()

collection = (
    ee.ImageCollection('MODIS/MCD43A4_006_NDVI')
    .filter(ee.Filter.date('2018-06-01', '2018-07-01'))
    .select("NDVI")
)
vis_params = {
    'min': 0.0,
    'max': 1.0,
    'palette': 'ndvi',
}

Map.add_time_slider(collection, vis_params, time_interval=2)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [15]:
Map.user_rois

In [16]:
images= collection.map(lambda img: img.clipToCollection(Map.user_rois))

In [17]:
Map = geemap.Map()

vis_params = {
    'min': 0.0,
    'max': 1.0,
    'palette': 'ndvi',
}

Map.add_time_slider(images, vis_params, time_interval=2)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [18]:
Map = geemap.Map()
fc = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(
    ee.Filter.eq('country_na', 'Netherlands')
)

Map.addLayer(fc, {'color': 'ff000000'}, "Netherlands")
Map.centerObject(fc)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [19]:
years = ee.List.sequence(2013, 2020)

In [20]:
def year_image(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = start_date.advance(1, 'year')
    collection= (
        ee.ImageCollection ('LANDSAT/LC08/C02/T1')
        .filterBounds(fc)
        .filterDate(start_date, end_date)
        
    )
    image= ee.Algorithms.Landsat.simpleComposite(collection).clipToCollection(fc)

    return image

In [21]:
images= years.map(year_image)

In [22]:
images

In [25]:
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'max': 128,
}

for index in range(0,8):
    image= ee.Image(images.get(index))
    layername= "Year" + str(index+2013)
    Map.addLayer(image, vis_params, layername)

Map

Map(bottom=11104.0, center=[52.130619595531755, 5.29281217801067], controls=(WidgetControl(options=['position'…